# Title

[Free Form Description]


**Data Sources:**

- `xxx.csv`: blablabla

**Data Output:**

- `xxx.csv`: blablabla

**Changes**

- 2019-02-18: Start project



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries,-load-data" data-toc-modified-id="Import-libraries,-load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries, load data</a></span></li></ul></div>

---

## Import libraries, load data

In [1]:
# Import libraries, get personalize boto3 client
import numpy as np
import pandas as pd
import json
import time

import boto3
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Display settings
from IPython.display import display
pd.options.display.max_columns = 100

In [16]:
# Load data
interactions_raw = pd.read_csv('data/raw/sales_total.csv', parse_dates=['Fakturadatum'])
items_raw = pd.read_csv('data/raw/artikel_agg_2018.csv')
users_raw = pd.read_csv('data/raw/customers_agg_2018.csv')

## Prepare, and Upload Training Data

Check [documentation] for more info. We prepare 3 datasets for
- users
- items
- interactions

The datasets are then prepared to match the columns in the schemas below.

In [3]:
"""Specify a Bucket and Data Output Location"""

bucket = "rbuerki-01"       # name of my S3 bucket
filename = "transactions_2018.csv"  # name to save the dataset under

In [11]:
"""Prepare interaction data"""

# Subset data for 2018 data only
interactions_18_full = interactions_raw.loc[interactions_raw['Fakturadatum'].dt.year == 2018]
interactions_18_part = interactions_18_full[['Kunde', 'Artikel', 'Fakturadatum', 'Nettowert']]

# Kick out all artikel that contain str values in their code
interactions_18_part['num'] = pd.to_numeric(interactions_18_part['Artikel'], errors='coerce')
interactions_18 = interactions_18_part.dropna(how='any')
interactions_18.drop(['num'], axis=1, inplace=True)

# Kick-out special customers
interactions = interactions_18.loc[interactions_18['Kunde'] > 700000]

C:\Users\r2d4\Anaconda3\envs\aws\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
# Rename Columns
interactions = interactions.rename(columns={'Kunde': 'USER_ID', 
                                            'Artikel': 'ITEM_ID',
                                            'Fakturadatum': 'TIMESTAMP',
                                            'Nettowert': 'EVENT_VALUE',
                                           })

# Check results
print(interactions.shape)
display(interactions.head(2))

(1402641, 4)


,USER_ID,ITEM_ID,TIMESTAMP,EVENT_VALUE
1388625,8488019,5171607,2018-01-03,77.3
1388626,8488019,5171101,2018-01-03,32.0


In [21]:
"""Prepare User data"""

users = users_raw[['Unnamed: 0', 'Branche']]
users = users.rename(columns={'Unnamed: 0': 'USER_ID', 
                              'Branche': 'BRANCHE',
                             })

# Check results
print(users.shape)
display(users.head(2))

(18625, 2)


,USER_ID,BRANCHE
0,8107232,15.0
1,8155006,10.0


In [23]:
"""Prepare Item data"""

items = items_raw[['id', 'name']]
items = items.rename(columns={'id': 'ITEM_ID', 
                              'name': 'NAME',
                             })

# Check results
print(items.shape)
display(items.head(2))

(73682, 2)


,ITEM_ID,NAME
0,0000351,Ankörn-Schablone MultiBlue für Montagepl
1,0002809,"Seitenrolle ø 30mm, max. 40kg Bauhöhe 31"


### Create Schemas

Schemas in Amazon Personalize are defined in the Avro format. For more information, see Apache Avro. The schema fields can be in any order but must match the order of the corresponding column headers in the imported data file. 